In [1]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
import os

In [2]:
import os
import pandas as pd
# ① 데이터 읽어오기
train_file = os.path.join(r'C:\DevRoot\Dropbox\Py03\09_chatbot\transaction\intent', "test2_data.xlsx")
data = pd.read_excel(train_file)
len(data)
len(data.dropna())
data=data.dropna()
data

,query,intent
0,안넝,0
1,ㅇㅏ ㄴ넝,0
2,아ㄴ넝,0
3,안ㄴㅓㅇ,0
4,안너ㅇ,0
...,...,...
5677,분실정보보여줘바바,6
5678,분실정보보자고요,6
5679,분실정보보자고용,6
5680,분실정보보자고영,6


In [3]:
data[data['intent'] == 5]

,query,intent
3628,ㅅㅂㄹㅁ,5
3629,ㅅㅐ,5
3630,ㅆㅂㄹㅁ,5
3631,ㅆㅍ,5
3632,ㅆㅣ,5
...,...,...
5210,ac발,5
5211,x대가리,5
5212,개같은년,5
5213,개좆같은년,5


In [4]:
# '질문(query)' 와 '의도(intent)' 
queries = data['query'].tolist()  
intents = data['intent'].tolist()

In [5]:
queries

['안넝',
 'ㅇㅏ ㄴ넝',
 '아ㄴ넝',
 '안ㄴㅓㅇ',
 '안너ㅇ',
 '얀뇽',
 'ㅇㅑㄴㄴㅛㅇ',
 '야ㄴㄴㅛㅇ',
 '얀ㄴㅛㅇ',
 '얀뇨ㅇ',
 '안녕해',
 'ㅇㅏㄴㄴㅕㅇㅎㅐ',
 '아ㄴㄴㅕㅇㅎㅐ',
 '안ㄴㅕㅇㅎㅐ',
 '안녀ㅇㅎㅐ',
 '안녕ㅎㅐ',
 '안녕하십니까',
 'ㅇㅏㄴㅕㅇㅎㅏㅅㅣㅂㄴㅣㄲㅏ',
 '아ㄴㄴㅕㅇㅎㅏㅅㅣㅂㄴㅣㄲㅏ',
 '안ㄴㅕㅇㅎㅏㅅㅣㅂㄴㅣㄲ ㅏ',
 '안녀ㅇㅎㅏㅅㅣㅂㄴㅣㄲㅏ',
 '안녕ㅎㅏㅅㅣㅂㄴㅣㄲㅏ',
 '안녕하ㅅㅣㅂㄴㅣㄲㅏ',
 '안녕하시ㅂㄴㅣㄲㅏ',
 '안녕하십ㄴㅣㄲㅏ',
 '안녕하십니ㄲㅏ',
 '안녕하시오',
 'ㅇㅏㄴㄴㅕㅇㅎㅏㅅㅣㅇㅗ',
 '아ㄴㄴㅕㅇㅎㅏㅅㅣㅇㅗ',
 '안ㄴㅕㅇㅎㅏㅅㅣㅇㅗ',
 '안녀ㅇㅎㅏㅅㅣㅇㅗ',
 '안녕ㅎㅏㅅㅣㅇㅗ',
 '안녕하ㅅㅣㅇㅗ',
 '안녕하시ㅇㅗ',
 '안녕이라고 해도되',
 'ㅇㅏㄴㄴㅕㅇㅇㅣㄹㅏㄱㅗㅎㅐㄷㅗㄷㅗㅣ',
 '아ㄴㄴㅕㅇㅇㅣㄹㅏㄱ ㅗㅎㅐㄷㅗㄷㅗㅣ',
 '안ㄴㅕㅇㅇㅣㄹㅏㄱㅗㅎㅐㄷㅗㄷㅗㅣ',
 '안녀ㅇㅇㅣㄹㅏㄱㅗㅎㅐㄷㅗㄷㅗㅣ',
 '안녕ㅇㅣㄹㅏㄱㅗㅎㅐㄷㅗㄷㅗㅣ',
 '안녕이ㄹㅏㄱㅗㅎㅐㄷㅗㄷㅗㅣ',
 '안녕이라ㄱㅗㅎㅐㄷㅗㄷㅗㅣ',
 '안녕이라고ㅎㅐㄷㅗㄷㅗㅣ',
 '안녕이라고해ㄷㅗㄷㅗㅣ',
 '안녕이라고해도ㄷㅗㅣ',
 '안녕이라고해도도ㅣ',
 '안녕이라할게',
 'ㅇㅏㄴㄴㅕㅇㅇㅣㄹㅏㅎㅏㄹㄱㅔ',
 '아ㄴㄴㅕㅇㅇㅣㄹㅏㅎㅏㄹㄱㅔ',
 '안ㄴㅕㅇㅇㅣㄹㅏㅎㅏㄹㄱㅔ',
 '안녀ㅇㅇㅣㄹㅏㅎㅏㄹㄱㅔ',
 '안녕ㅇㅣㄹㅏㅎㅏㄹㄱㅔ',
 '안녕이ㄹㅏㅎㅏㄹㄱㅔ',
 '안녕이라ㅎㅏㄹㄱㅔ',
 '안녕이라하ㄹㄱㅔ',
 '안녕이라할ㄱㅔ',
 '안뇨하세여',
 'ㅇㅏㄴㄴㅛㅎㅏㅅㅔㅇㅕ',
 '아ㄴㄴㅛㅎㅏㅅㅔㅇㅕ',
 '안ㄴㅛㅎㅏㅅㅔ ㅇㅕ',
 '안뇨ㅎㅏㅅㅔㅇㅕ',
 '안뇨하ㅅㅔㅇㅕ',
 '안뇨하세ㅇㅕ',
 '안녕하세요',
 'ㅇㅏ ㄴㄴㅕㅇㅎㅏㅅㅔㅇㅛ',
 '아ㄴㄴㅕㅇㅎㅏㅅㅔㅇㅛ',
 '안ㄴㅕㅇㅎㅏㅅㅔㅇㅛ',
 '안녀ㅇㅎㅏㅅㅔㅇㅛ',
 '안녕ㅎㅏㅅㅔㅇㅛ',
 '안녕하ㅅㅔㅇㅛ',
 '안녕하세ㅇㅛ',
 '안녀하세요

In [6]:
# 전처리 모듈 불러오기
from chatbot.utils.Preprocess import Preprocess
p = Preprocess(word2index_dic=os.path.join(r'C:\DevRoot\Dropbox\Py03\09_chatbot\transaction\intent', 'chatbot_dict.bin'),
               userdic=os.path.join(r'C:\DevRoot\Dropbox\Py03\09_chatbot\transaction\intent\dict', 'user_dic.tsv'))



In [7]:
# 단어 시퀀스 생성 (가장 시간 많이 걸림)
# 해당 단어에 매칭되는 번호로 시퀀스 생성

# ★ 시간 제법 걸림 * 몇십초 정도..

sequences = []
for sentence in queries:
    pos = p.pos(sentence)             # 품사지정, 배열
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)

    sequences.append(seq)


In [8]:
# 결과 비교
max(len(l) for l in queries)


225

In [9]:
sequences[:20]

[[1],
 [1, 1],
 [707],
 [1],
 [709],
 [710],
 [710],
 [1],
 [710],
 [714],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [721],
 [1],
 [721],
 [1, 136]]

In [10]:
len(queries)

5668

In [11]:
len(sequences)

5668

In [12]:
# 단어 시퀀스 벡터 크기
MAX_SEQ_LEN = 15


def GlobalParams():
    global MAX_SEQ_LEN

In [13]:
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

print(padded_seqs.shape)
print(len(intents))

(5668, 15)
5668


In [14]:
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents)) # 패딩처리된 시퀀스와 의도(intent) 리스트 전체를 데이터셋 객체로
ds = ds.shuffle(len(queries)) # 랜덤 섞기

# 학습셋:검증셋:테스트셋 = 7:2:1
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 7
VOCAB_SIZE = len(p.word_index) + 1 #전체 단어 개수

In [15]:
# ④ CNN 모델 정의
# keras 함수형 모델 방식으로 구현
input_layer = Input(shape=(MAX_SEQ_LEN,))  # 입려크기
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(7, name='logits')(dropout_hidden)  # 최종적으로 5가의 의도 클래스를 분류. 결과로 나온 값(logits) 을을 점수(score) 라 부른다
predictions = Dense(7, activation=tf.nn.softmax)(logits)


In [16]:
# ⑤ 모델 생성 
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [17]:
# ⑥ 모델 학습

# ★ 시간 걸림 ★  

model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/7
199/199 [==============================] - 9s 27ms/step - loss: 1.2139 - accuracy: 0.5750 - val_loss: 0.9603 - val_accuracy: 0.6637
Epoch 2/7
199/199 [==============================] - 4s 23ms/step - loss: 0.7750 - accuracy: 0.7404 - val_loss: 0.5904 - val_accuracy: 0.7899
Epoch 3/7
199/199 [==============================] - 4s 21ms/step - loss: 0.6000 - accuracy: 0.7918 - val_loss: 0.4741 - val_accuracy: 0.8252
Epoch 4/7
199/199 [==============================] - 4s 20ms/step - loss: 0.5064 - accuracy: 0.8142 - val_loss: 0.4285 - val_accuracy: 0.8376
Epoch 5/7
199/199 [==============================] - 4s 22ms/step - loss: 0.4446 - accuracy: 0.8349 - val_loss: 0.3973 - val_accuracy: 0.8447
Epoch 6/7
199/199 [==============================] - 5s 23ms/step - loss: 0.4116 - accuracy: 0.8462 - val_loss: 0.4234 - val_accuracy: 0.8376
Epoch 7/7
199/199 [==============================] - 4s 22ms/step - loss: 0.4030 - accuracy: 0.8394 - val_loss: 0.3739 - val_accuracy: 0.8526


In [18]:
# ⑦ 모델 평가(테스트 데이터 셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

29/29 [==============================] - 0s 4ms/step - loss: 0.3607 - accuracy: 0.8657
Accuracy: 86.572438
loss: 0.360661


In [19]:
# ⑧ 모델 저장
model.save(os.path.join(r'C:\DevRoot\Dropbox\Py03\09_chatbot\transaction\intent','intent_test_model.h5'))

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing


# 의도 분류 모델 모듈
class IntentModel:
    def __init__(self, model_name, preprocess):

        # 의도 클래스 별 레이블
        self.labels = {0: "인사", 1: "조회", 2: "이체", 3: "납부", 4: "답변", 5: "욕설", 6: "분실"}

        # 의도 분류 모델 불러오기
        self.model = load_model(model_name)

        # 챗봇 Preprocess 객체
        self.p = preprocess


    # 의도 클래스 예측
    def predict_class(self, query):   # query 문장 입력
        # 형태소 분석
        pos = self.p.pos(query)

        # 문장내 키워드 추출(불용어 제거)
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 단어 시퀀스 벡터 크기
        MAX_SEQ_LEN = 15

        # 패딩처리
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

        predict = self.model.predict(padded_seqs)
        predict_class = tf.math.argmax(predict, axis=1)
        return predict_class.numpy()[0]





## 의도분류 모델 사용 테스트

In [21]:
from chatbot.utils.Preprocess import Preprocess
from chatbot.models.intent.intentModel1 import IntentModel
import os

In [22]:
p = Preprocess(word2index_dic=os.path.join(r'C:\DevRoot\Dropbox\Py03\09_chatbot\transaction\intent',"chatbot_dict.bin"),
               userdic=os.path.join(r'C:\Users\drago\OneDrive\문서\DevRoot\dataset\chatbot\user_dic.tsv'))

In [23]:
intent = IntentModel(model_name=os.path.join(r'C:\DevRoot\Dropbox\Py03\09_chatbot\transaction\intent','intent_test_model.h5'), preprocess=p)

In [24]:
query = "안녕"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]

print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

1/1 [==============================] - 0s 435ms/step
안녕
의도 예측 클래스 :  0
의도 예측 레이블 :  인사


In [25]:
query = "적금정보조회점하고싶어"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]

print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

1/1 [==============================] - 0s 37ms/step
적금정보조회점하고싶어
의도 예측 클래스 :  1
의도 예측 레이블 :  조회


In [26]:
query = "야돈보내겠다니까요"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]

print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

1/1 [==============================] - 0s 35ms/step
야돈보내겠다니까요
의도 예측 클래스 :  2
의도 예측 레이블 :  이체


In [27]:
query = "씨발"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]

print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

1/1 [==============================] - 0s 33ms/step
씨발
의도 예측 클래스 :  5
의도 예측 레이블 :  욕설


In [28]:
query = "적금"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]

print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

1/1 [==============================] - 0s 34ms/step
적금
의도 예측 클래스 :  1
의도 예측 레이블 :  조회


In [29]:
query = "분실확인"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]

print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

1/1 [==============================] - 0s 35ms/step
분실확인
의도 예측 클래스 :  1
의도 예측 레이블 :  조회
